# Live Input and Output
## Connecting to a SpiNNaker board
<img src="SpiNNakerPorts.png" />

Communication with the outside world from a SpiNNaker simulation can be done using one of the following:
- SpiNNaker Link connection.  This is a direct connection in to the router of one of the chips and as such can operate at the full bandwidth of the router.  There are 2 of these connections on a 4-node board and 1 on each 48-node board.  All of these can be used in a multi-board configuration, but the FPGA connection on the same link *must be disabled* in this case, as this can otherwise overload and physically break the link.
- SpiNN-Link connection.  This is a connection to the SATA-connectors on the board, which then interfaces with the FPGAs on the board.  This connection can operate at the speed of the interface between the boards, and has access to multiple chips on the board-side of the connection.  There are 9 such connectors on each 48-node board, though 6 of those are used for the connections between boards in multi-board machines.  This requires the FPGA that the link is connected to to be configured to forward traffic to and from the chips on the board.  This is *not* performed by the software so additional development is required here.
- Ethernet connection.  This connects via the monitor core of the first chip on each board.  As such it is the slowest of the connections, but as it works hardware that most PCs have, it is the easiest to use.

## Communicating over Ethernet
The SpiNNaker software extends PyNN by allowing live input into and live output out of a neural network running on the platform.  The SpiNNaker software provides a number of different ways for a PyNN simulation to communicate.  These are described below.  In all cases the external agent can discover information about a running simulation through a database that can be written before the simulation starts executing on the machine; this includes the placement of the parts of a Population on the machine, and the multicast keys which are being used to send spikes and/or commands, allowing the agent to decode information being received, or encode information being sent.

External agents can register to receive notifications from the software when the database has been written, when the simulation starts (or resumes after a pause), and when the simulation ends (or is paused).  The software will send these notifications as UDP messages.  After the database message, the agent is expected to reply to indicate that it is ready for the simulation to start; the software will pause until this has been received to ensure that the agent is ready.

A number of "Connection" implementations are provided to help in the construction of the external agent.  These can be requested to call functions when certain events occur, such as when the simulation is starting or finishing, or when a message has been received from the simulation.  The port on which these connections listen for the notification messages from the simulation software can be registered during the calls made as the simulation is set up.  These details will also be discussed below.

The normal order of operation to set up an external communication is as follows:
1. Set up the "external agent" connections so that they are ready to listen for notification messages from the software.  These may have to be executed in a separate thread or process, but the connection implementations take care of this.
1. During the PyNN simulation specification, ensure that the connection details are registered with the software.
1. Run the simulation.

### Live Spike Input - SpikeInjector

<img src="SpikeInjector.png">

Spikes can be sent into a PyNN simulation by using a ```SpikeInjector``` as the model of a Population.  This model expects to receive SpiNNaker Datagram Protocol (SDP) messages containing spikes.  A ```SpynnakerLiveSpikesConnection``` can be used by an external agent to send the ```SpikeInjector``` these messages without having to understand the details.  Note that a single connection can be used to send spikes to multiple sources.

The following is an example of live spike transmission:

In [1]:
# Import the simulator
import spynnaker.pyNN as sim

# Other imports used in this example
from time import sleep


# Set up a function that will start sending spikes when the simulation starts.
# This is automatically run in a separate thread from the rest of the simulation.
# This must accept two arguments: 
#    The label of the Population that the callback is registered against, 
#    and the connection the callback is registered against.
def send_spikes(label, connection):
    # Sleep to make sure everything is fully ready
    sleep(0.01)
    
    # Send a spike to neuron 0
    connection.send_spike(label, 0)
    
    # Send spikes to neurons 1-4 after 0.1ms
    sleep(0.1)
    connection.send_spikes(label, range(1, 5))


# Keep track of the label of the injector as this needs to match up in several places
injector_label = "injector"

# Create the connection, noting that the label will be a "sender".
# Note the use of local_port=None allows the automatic assignment of a port.
connection = sim.external_devices.SpynnakerLiveSpikesConnection(
    local_port=None, send_labels=[injector_label])

# Add a callback to be called at the start of the simulation
connection.add_start_resume_callback(injector_label, send_spikes)

# Set up the simulation itself
sim.setup(1.0)

# Set up the injector population with 5 neurons, 
# simultaneously registering the connection as a listener
injector = sim.Population(
    5, sim.external_devices.SpikeInjector(
        database_notify_port_num=connection.local_port),
    # Critical: Make sure the label is used!
    label=injector_label)

# Set up a Population to receive spikes and record
pop = sim.Population(5, sim.IF_curr_exp(), label="pop")
pop.record("spikes")

# Connect the injector to the population
sim.Projection(injector, pop, sim.OneToOneConnector(), sim.StaticSynapse(weight=5))

# Run the simulation and get the spikes out
sim.run(1000)
spikes = pop.get_data("spikes").segments[0].spiketrains

# End the simulation and display the spikes
sim.end()
print(spikes)


2024-01-16 17:28:54 INFO: Read configs files: /home/bbpnrsoa/sPyNNakerGit/SpiNNUtils/spinn_utilities/spinn_utilities.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNMachine/spinn_machine/spinn_machine.cfg, /home/bbpnrsoa/sPyNNakerGit/PACMAN/pacman/pacman.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNMan/spinnman/spinnman.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg, /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg, /home/bbpnrsoa/.spynnaker.cfg
2024-01-16 17:28:54 INFO: Will search these locations for binaries: /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/model_binaries
2024-01-16 17:28:55 WARNING: /home/bbpnrsoa/reports/reports has 75 old reports that have not been closed
2024-01-16 17:28:55 INFO: Setting hardware timestep as 1000 microseconds based on simulation time step of 1000 and timescale factor of 1.0
2024-01-16 17:28:55 INFO: Starting execution process
2024-01-16 17:28:55 INFO: Simulating for 1000 1.0 ms timesteps 

['/home/bbpnrsoa/sPyNNakerGit/SpiNNUtils/spinn_utilities/spinn_utilities.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNMachine/spinn_machine/spinn_machine.cfg', '/home/bbpnrsoa/sPyNNakerGit/PACMAN/pacman/pacman.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNMan/spinnman/spinnman.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg', '/home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg', '/home/bbpnrsoa/.spynnaker.cfg']


2024-01-16 17:28:55 INFO: SpYNNakerNeuronGraphNetworkSpecificationReport skipped as cfg Reports:write_network_graph is False
2024-01-16 17:28:55 INFO: Network Specification report took 0:00:00.000762 
2024-01-16 17:28:55 INFO: Splitter reset took 0:00:00.000036 
Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2024-01-16 17:28:55 INFO: Spynnaker splitter selector took 0:00:00.028546 
Adding delay extensions as required
|0%                          50%                         100%|
2024-01-16 17:28:55 INFO: DelaySupportAdder took 0:00:00.023509 
Partitioning Graph
|0%                          50%                         100%|
2024-01-16 17:28:55 INFO: Splitter partitioner took 0:00:00.016838 
2024-01-16 17:28:55 INFO: 0.02 Boards Required for 1 chips
2024-01-16 17:28:55 INFO: Requesting job with 1 boards
Created spalloc job 379108
2024-01-16 17:28:55 INFO: Created spalloc job 379108
Job has been queued by the spalloc server.
2024

[<SpikeTrain(array([21.]) * ms, [0.0 ms, 1000.0 ms])>, <SpikeTrain(array([122.]) * ms, [0.0 ms, 1000.0 ms])>, <SpikeTrain(array([122.]) * ms, [0.0 ms, 1000.0 ms])>, <SpikeTrain(array([122.]) * ms, [0.0 ms, 1000.0 ms])>, <SpikeTrain(array([122.]) * ms, [0.0 ms, 1000.0 ms])>]


### Live Spike Output - activate_live_output_for

<img src="SpikeOutput.png">

Spikes can be received from a PyNN simulation by calling a special function ```activate_live_output_for```, passing in the population from which the spikes are to be received, as well as other parameters to specify the properties of the communication. This will cause the simulation to send SpiNNaker Datagram Protocol (SDP) messages containing spikes from the machine during the simulation. Again, a ```SpynnakerLiveSpikesConnection``` can be used by an external agent to receive these messages without having to understand the details.  Note that a single connection can be used to receive spikes from multiple sources, and can also send spikes as described in the previous section.

The following is an example of live spike output:

In [2]:
# Import the simulator
import spynnaker.pyNN as sim

# Set up a function that will receive spikes.
# This is automatically run in a separate thread from the rest of the simulation.
# This must accept three arguments: 
#    The label of the Population that the callback is registered against, 
#    the time at which a spikes are received, in units of simulation time step,
#    and the neuron ids that spiked at this time.
def receive_spikes(label, time, neuron_ids):
    print("Received spikes {} from {} at time {}".format(neuron_ids, label, time))


# Keep track of the label of the Population
pop_label = "pop"

# Create the connection, noting that the label will be a "receiver".
# Note the use of local_port=None allows the automatic assignment of a port.
connection = sim.external_devices.SpynnakerLiveSpikesConnection(
    local_port=None, receive_labels=[pop_label])

# Add a callback to be called when spikes are received
connection.add_receive_callback(pop_label, receive_spikes)

# Set up the simulation itself
sim.setup(1.0)

# Set up a population which will spike
pop = sim.Population(
    5, sim.SpikeSourcePoisson(rate=10),
    # Critical: Make sure the label is used!
    label=pop_label)

# Activate the live output, 
# simultaneously registering the connection as a listener
sim.external_devices.activate_live_output_for(
    pop, database_notify_port_num=connection.local_port)

# Run the simulation
sim.run(1000)

# End the simulation
sim.end()


2024-01-16 17:29:35 INFO: 0.0.0.0:43906 Waiting for message to indicate that the database is ready
2024-01-16 17:29:35 INFO: Receive callback <function receive_spikes at 0x7f790d6dbee0> registered to label pop
2024-01-16 17:29:35 INFO: Read configs files: /home/bbpnrsoa/sPyNNakerGit/SpiNNUtils/spinn_utilities/spinn_utilities.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNMachine/spinn_machine/spinn_machine.cfg, /home/bbpnrsoa/sPyNNakerGit/PACMAN/pacman/pacman.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNMan/spinnman/spinnman.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg, /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg, /home/bbpnrsoa/.spynnaker.cfg
2024-01-16 17:29:35 INFO: Will search these locations for binaries: /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/model_binaries : /home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/common_model_binaries
2024-01-16 17:29:35 WARNING: /home/bbpnrsoa/reports/rep

['/home/bbpnrsoa/sPyNNakerGit/SpiNNUtils/spinn_utilities/spinn_utilities.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNMachine/spinn_machine/spinn_machine.cfg', '/home/bbpnrsoa/sPyNNakerGit/PACMAN/pacman/pacman.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNMan/spinnman/spinnman.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg', '/home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg', '/home/bbpnrsoa/.spynnaker.cfg']


2024-01-16 17:29:35 INFO: SpYNNakerNeuronGraphNetworkSpecificationReport skipped as cfg Reports:write_network_graph is False
2024-01-16 17:29:35 INFO: Network Specification report took 0:00:00.000818 
2024-01-16 17:29:35 INFO: Splitter reset took 0:00:00.000069 
Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2024-01-16 17:29:35 INFO: Spynnaker splitter selector took 0:00:00.016140 
Adding delay extensions as required
|0%                          50%                         100%|
2024-01-16 17:29:35 INFO: DelaySupportAdder took 0:00:00.006049 
Partitioning Graph
|0%                          50%                         100%|
2024-01-16 17:29:35 INFO: Splitter partitioner took 0:00:00.065995 
2024-01-16 17:29:35 INFO: 0.02 Boards Required for 1 chips
2024-01-16 17:29:35 INFO: Requesting job with 1 boards
Created spalloc job 379109
2024-01-16 17:29:35 INFO: Created spalloc job 379109
Job has been queued by the spalloc server.
2024

Received spikes [2] from pop at time 12
Received spikes [4] from pop at time 31
Received spikes [3] from pop at time 49
Received spikes [2] from pop at time 66
Received spikes [4] from pop at time 72
Received spikes [1] from pop at time 77
Received spikes [2] from pop at time 131
Received spikes [0] from pop at time 133
Received spikes [3] from pop at time 147
Received spikes [4] from pop at time 171
Received spikes [3] from pop at time 209
Received spikes [0] from pop at time 218
Received spikes [0] from pop at time 237
Received spikes [0] from pop at time 244
Received spikes [3] from pop at time 259
Received spikes [2] from pop at time 266
Received spikes [1, 2, 3] from pop at time 275
Received spikes [0] from pop at time 303
Received spikes [4] from pop at time 311
Received spikes [1] from pop at time 345
Received spikes [3] from pop at time 351
Received spikes [1] from pop at time 352
Received spikes [4] from pop at time 355
Received spikes [3] from pop at time 356
Received spikes 

2024-01-16 17:30:11 INFO: ** Sending pause / stop message to external sources to state the simulation has been paused or stopped. **
2024-01-16 17:30:11 INFO: Application runner took 0:00:01.960115 
2024-01-16 17:30:11 INFO: Extract IO buff skipped as cfg Reports:extract_iobuf is False
2024-01-16 17:30:11 INFO: Starting buffer extraction using Java
2024-01-16 17:30:11 INFO: Buffer extractor took 0:00:00.004939 
clearing IOBUF from the machine
|0%                          50%                         100%|
2024-01-16 17:30:11 INFO: Clear IO buffer took 0:00:00.016992 
Getting provenance data from application graph
|0%                          50%                         100%|
2024-01-16 17:30:11 INFO: Graph provenance gatherer took 0:00:00.003939 
Getting provenance data
|0%                          50%                         100%|
2024-01-16 17:30:11 INFO: Placements provenance gatherer took 0:00:00.293761 
Getting Router Provenance
|0%                          50%                     

### Live Rate Input - add_poisson_live_rate_control

<img src="PoissonControl.png">

The spike rate of a ```SpikeSourcePoisson``` can be controlled during the simulation, by calling a function ```add_poisson_live_rate_control```, passing in the Poisson source to be controlled, as well as other parameters of the communication if required.  This will set up a mechanism by which the Poisson rates can be changed by sending SDP packets containing multicast keys and rates.  A ```SpynnakerPoissonControlConnection``` can by used by the external agent to send these packets without requiring knowledge of this mechanism.  Note that multiple sources can be controlled with the same connection.

The software on SpiNNaker makes some assumptions using the rates of Poisson spikes sources, therefore it is important to tell the source what the maximum rate to expect will be.  This is done by providing a value for ```max_rate``` using the  ```additional_parameters``` of the Population of the source.  This may or may not be critical for the functioning of the network depending on how high the rate goes, and whether or not the source is being recorded; in particular space for recording will be reserved based on this value, so if the rate increases during simulation, recorded values may be lost.

The following is an example of live Poisson rate control:

In [3]:
# Import the simulator
import spynnaker.pyNN as sim

# Other imports used in this example
from time import sleep
import pyNN.utility.plotting as plot


# Set up a function that will start setting rates when the simulation starts.
# This is automatically run in a separate thread from the rest of the simulation.
# This must accept two arguments: 
#    The label of the Population that the callback is registered against, 
#    and the connection the callback is registered against.
def set_rates(label, connection):
    try:
        # Set the rate of neuron 0 to 10Hz at about 100ms
        sleep(0.1)
        print("Setting rate of 0")
        connection.set_rate(label, 0, 10)

        # Set the rate of neurons 1-4 to 50Hz at about 500ms
        sleep(0.4)
        print("Setting rate of 1-4")
        connection.set_rates(label, [(i, 50) for i in range(1, 5)])
    except Exception:
        import traceback
        traceback.print_exc()


# Keep track of the label of the Population
pop_label = "pop"

# Create the connection, noting that the label will be the controlled source.
# Note the use of local_port=None allows the automatic assignment of a port.
connection = sim.external_devices.SpynnakerPoissonControlConnection(
    local_port=None, poisson_labels=[pop_label])

# Add a callback to be called at the start of the simulation
connection.add_start_resume_callback(pop_label, set_rates)

# Set up the simulation itself
sim.setup(1.0)

# Set up a population which will spike
pop = sim.Population(
    5, sim.SpikeSourcePoisson(rate=0),
    # Provide an additional argument to indicate that the rate will change
    additional_parameters={'max_rate': 50},
    # Critical: Make sure the label is used!
    label=pop_label)

# Record the population
pop.record("spikes")

# Specify that this Poisson source is to be controlled,
# simultaneously registering the connection as a listener
sim.external_devices.add_poisson_live_rate_control(
    pop, database_notify_port_num=connection.local_port)

# Run the simulation and get the spikes
sim.run(1000)
spikes = pop.get_data("spikes").segments[0].spiketrains

# End the simulation and plot the spikes
sim.end()
plot.Figure(
    plot.Panel(spikes, yticks=True, markersize=5, xlim=(0, 1000)),
    title="Poisson Spikes"
)

2024-01-16 17:30:11 INFO: 0.0.0.0:35610 Waiting for message to indicate that the database is ready
2024-01-16 17:30:11 INFO: Read configs files: /home/bbpnrsoa/sPyNNakerGit/SpiNNUtils/spinn_utilities/spinn_utilities.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNMachine/spinn_machine/spinn_machine.cfg, /home/bbpnrsoa/sPyNNakerGit/PACMAN/pacman/pacman.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNMan/spinnman/spinnman.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg, /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg, /home/bbpnrsoa/.spynnaker.cfg
2024-01-16 17:30:11 INFO: Will search these locations for binaries: /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/model_binaries : /home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/common_model_binaries
2024-01-16 17:30:12 WARNING: /home/bbpnrsoa/reports/reports has 76 old reports that have not been closed
2024-01-16 17:30:12 INFO: Setting hardware timestep as 1000 m

['/home/bbpnrsoa/sPyNNakerGit/SpiNNUtils/spinn_utilities/spinn_utilities.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNMachine/spinn_machine/spinn_machine.cfg', '/home/bbpnrsoa/sPyNNakerGit/PACMAN/pacman/pacman.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNMan/spinnman/spinnman.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg', '/home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg', '/home/bbpnrsoa/.spynnaker.cfg']


2024-01-16 17:30:12 INFO: SpYNNakerNeuronGraphNetworkSpecificationReport skipped as cfg Reports:write_network_graph is False
2024-01-16 17:30:12 INFO: Network Specification report took 0:00:00.000697 
2024-01-16 17:30:12 INFO: Splitter reset took 0:00:00.000030 
Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2024-01-16 17:30:12 INFO: Spynnaker splitter selector took 0:00:00.020629 
Adding delay extensions as required
|0%                          50%                         100%|
2024-01-16 17:30:12 INFO: DelaySupportAdder took 0:00:00.008327 
Partitioning Graph
|0%                          50%                         100%|
2024-01-16 17:30:12 INFO: Splitter partitioner took 0:00:00.004865 
2024-01-16 17:30:12 INFO: 0.02 Boards Required for 1 chips
2024-01-16 17:30:12 INFO: Requesting job with 1 boards
Created spalloc job 379110
2024-01-16 17:30:12 INFO: Created spalloc job 379110
Job has been queued by the spalloc server.
2024

Setting rate of 0
Setting rate of 1-4


2024-01-16 17:30:49 INFO: ** Sending pause / stop message to external sources to state the simulation has been paused or stopped. **
2024-01-16 17:30:49 INFO: Application runner took 0:00:01.230273 
2024-01-16 17:30:49 INFO: Extract IO buff skipped as cfg Reports:extract_iobuf is False
2024-01-16 17:30:49 INFO: Starting buffer extraction using Java
2024-01-16 17:30:53 INFO: Buffer extractor took 0:00:03.680122 
clearing IOBUF from the machine
|0%                          50%                         100%|
2024-01-16 17:30:53 INFO: Clear IO buffer took 0:00:00.024748 
Getting provenance data from application graph
|0%                          50%                         100%|
2024-01-16 17:30:53 INFO: Graph provenance gatherer took 0:00:00.007696 
Getting provenance data
|0%                          50%                         100%|
2024-01-16 17:30:53 INFO: Placements provenance gatherer took 0:00:00.308782 
Getting Router Provenance
|0%                          50%                     

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Live Voltage Output - Robot Control

<img src="VoltageOutput.png">

The software provides a specific neuron model, ```ExternalDeviceLifControl```, which is designed to be used to output samples of the membrane voltage of the neuron during simulation.  This could be used, for example, to control the speed of a motor in a robot.  The values are again contained in SDP packets containing pairs of multicast keys and payloads, which are described as commands.  This additionally differs from the spike output in that the multicast keys can be specified by the user rather than being assigned by the software; for this reason some care must be taken not to use the same key in multiple different places (though the software will check this and report an error if encountered).  It is also possible to configure other aspects of the communication, such as how often the value is transmitted, the minimum and maximum values, and even the binary format of the value.  These parameters are encapsulated in the ```AbstractMulticastControllableDevice```, a set of which are provided to the ```ExternalDeviceLifControl```.

The software also provides an ```EthernetControlConnection``` to allow the decoding of these packets by an external agent.  This is different from the other connections in that it also takes an ```AbstractEthernetTranslator```, which is asked to translate each received message.  This might, for example, convert the message into a command for a robot.

To avoid the complexities of building the connection, population, and all the other parts required, an ```EthernetControlPopulation``` is provided.  This will return a ```Population``` object which can be used as the target of a ```Projection```.

An example of this interaction is shown below:

In [4]:
# Import the simulator
import spynnaker.pyNN as sim

# Import the extra classes
from spinn_front_end_common.interface.ds import DataType
from spynnaker.pyNN.external_devices_models import (
    AbstractEthernetTranslator, AbstractMulticastControllableDevice)
from spynnaker.pyNN.external_devices_models\
    .abstract_multicast_controllable_device import SendType


class TestTranslator(AbstractEthernetTranslator):
    
    def translate_control_packet(self, packet):
        print("Received key={}, voltage={}".format(
            packet.key, packet.payload / DataType.S1615.max))
    

class TestDevice(AbstractMulticastControllableDevice):
    
    @property
    def device_control_partition_id(self):
        # This should be unique to the device, but is otherwise unimportant
        return "Test"
    
    @property
    def device_control_key(self):
        # This should be unique to the device
        return 0
    
    @property
    def device_control_uses_payload(self):
        # This returns True to receive the voltage, 
        # or False if only the key is desired
        return True
    
    @property
    def device_control_min_value(self):
        # Return the minimum value accepted by the device.  If the membrane
        # voltage is below this value, this value will be used.
        return 0
    
    @property
    def device_control_max_value(self):
        # Return the maximum value accepted by the device.  If the membrane
        # voltage is above this value, this value will be used.
        return 100
    
    @property
    def device_control_timesteps_between_sending(self):
        # The number of timesteps between sending values.  Controls the
        # update rate of the value.
        return 10
    
    @property
    def device_control_send_type(self):
        # The type of the value - one of the SendType values
        return SendType.SEND_TYPE_ACCUM
    
    @property
    def device_control_scaling_factor(self):
        # The amount to multiply the voltage by before transmission
        return 1.0
    

# Setup the simulation
sim.setup(1.0)

# Create some stimulation
stim = sim.Population(1, sim.SpikeSourcePoisson(rate=50), label="stim")

# Create the model that will generate the voltage
pop = sim.external_devices.EthernetControlPopulation(
    n_neurons=1,
    model=sim.external_devices.ExternalDeviceLifControl(
        devices=[TestDevice()],
        create_edges=False,
        translator=TestTranslator()),
    label="test")

# Connect the stimulation to the population
sim.Projection(stim, pop, sim.OneToOneConnector(), sim.StaticSynapse(weight=1))

# Run the simulation then stop
sim.run(1000)
sim.end()


ModuleNotFoundError: No module named 'data_specification'

## Excercises
The following exercises are designed to enhance your understanding of the use of Live I/O with SpiNNaker.  You have the option of following the step-by-step instructions in the linked notebooks, or you can try to follow the instructions below each task (this will be harder).

 - [Task 1: Send data (Easy)](task1.ipynb)
    - Create a network with a timestep of 1.0ms consisting of 100 LIF neurons which are randomly connected to each other with a probability of 0.08, using a weight of 0.5nA and a delay of 2ms and which are all stimulated from a single SpikeInjector neuron with a weight of 5nA and a delay of 1.0ms. Send spikes into the network using the injector at approximately 100ms intervals.  Record the LIF neurons and run for 1000ms, plotting the results after the simulation has completed.
 
 - [Task 2: Receive data (Easy)](task2.ipynb)
    - Create a network with a timestep of 1.0ms consisting of 100 Poisson neurons connected with a probability of 0.1 to 100 LIF neurons, with a weight of 0.5nA and a delay of 1ms.  Don't record the Population; instead receive the spikes live from the Population and print out when each spike or spikes arrive.
    
 - [Task 3: Rate Changes (Easy)](task3.ipynb)
     - Using the network described in Task 2 above, record the LIF neurons rather than extracting the live spikes.  Instead, set the Poisson neurons to 0Hz initially but allow them to have a maximum rate of 100Hz.  Change the rate of the Poisson sources during the simulation progressively to 10Hz, 20Hz, 50Hz and 100Hz with 0.2 seconds in between each.  Plot the results after the simulation has completed.
 
 - [Task 4: Receive Voltage (Moderate)](task4.ipynb)
     - Create a network with a timestep of 1.0ms consisting of 100 Poisson neurons all connected to a single neuron external control Population using a weight of 0.1nA and a delay of 1ms.  Create a device that sends the voltage using integers in the range -50 to 50mV every 5ms using a key of 10.  Create a translator that receives the voltage and prints out the value.
 
    